In [18]:
import os
import random
import numpy as np
import torch
from transformers import BertTokenizer, BertModel, LlamaTokenizer, LlamaModel
from sklearn.metrics.pairwise import cosine_similarity
import json
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
import re

cpu


In [19]:
#TODO
#1 TRY ADDING BYLINE AND HEADER
#2 TRY fine-tuning model somehow

In [20]:
#from google.colab import drive
#drive.mount('/content/drive')

In [21]:
#hyperparameters
g_batch_size = 256

In [22]:
RandomSeed = 52
random.seed(RandomSeed)
torch.manual_seed(RandomSeed)

In [23]:
model_name  = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.to(device)

#model_name = "meta-llama/Llama-3.2-1B"
#tokenizer = LlamaTokenizer.from_pretrained(model_name)
#model = LlamaModel.from_pretrained(model_name)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [24]:
def get_document_embedding(text):
    # Tokenize the input text
    inputs = tokenizer.batch_encode_plus(
            text,
            padding=True,
            truncation=True,
            return_tensors='pt',
            add_special_tokens=True)

    # Move the inputs to the appropriate device (e.g., GPU if available)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Get the hidden states from the model without gradients
    with torch.no_grad():
        outputs = model(**inputs)

    # Use mean pooling over token embeddings to get a single document embedding
    mean_pooled_embedding = outputs.last_hidden_state.mean(dim=1).cpu().numpy()  # Move tensor to CPU and convert to NumPy
    return mean_pooled_embedding


In [25]:
def read_files_dataset(file_path):
    documents = []
    try:
        # Try reading the file with utf-8 encoding, if it fails, fall back to ISO-8859-1
        with open(file_path, 'r', encoding='utf-8') as file:
            document = {}
            in_text = False
            in_headline = False
            in_byline = False
            in_doc = False

            for line in file:
                line = line.strip()

                # Check if it's the start of a new document
                if line.startswith("<DOCNO>"):
                    document['documentID'] = re.sub(r'<.*?>', '', line).strip()
                elif line.startswith("<HEADLINE>"):
                    in_headline = True
                    headline_parts = []
                    while in_headline:
                        line = next(file).strip()
                        if line.startswith("</HEADLINE>"):
                            in_headline = False
                        else:
                            headline_parts.append(re.sub(r'<.*?>', '', line))
                    #document['headline'] = ' '.join(headline_parts).strip()
                elif line.startswith("<BYLINE>"):
                    in_byline = True
                    byline_parts = []
                    while in_byline:
                        line = next(file).strip()
                        if line.startswith("</BYLINE>"):
                            in_byline = False
                        else:
                            byline_parts.append(re.sub(r'<.*?>', '', line))
                    #document['byLine'] = ' '.join(byline_parts).strip()
                elif line.startswith("<TEXT>"):
                    in_text = True
                    text_parts = []
                    while in_text:
                        line = next(file).strip()
                        if line.startswith("</TEXT>"):
                            in_text = False
                        else:
                            text_parts.append(re.sub(r'<.*?>', '', line))
                    document['text'] = ' '.join(text_parts).strip()

                # End of a document
                elif line.startswith("</DOC>"):
                    documents.append(document)
                    document = {}

    except UnicodeDecodeError as e:
        documents=[]
        print(f"UnicodeDecodeError: {e}")
        print(f"Trying ISO-8859-1 encoding for file: {file_path}")
        try:
            # Try opening the file with ISO-8859-1 encoding if utf-8 fails
            with open(file_path, 'r', encoding='ISO-8859-1') as file:
                document = {}
                in_text = False
                in_headline = False
                in_byline = False
                in_doc = False

                for line in file:
                    line = line.strip()

                    # Check if it's the start of a new document
                    if line.startswith("<DOCNO>"):
                        document['documentID'] = re.sub(r'<.*?>', '', line).strip()
                    elif line.startswith("<HEADLINE>"):
                        in_headline = True
                        headline_parts = []
                        while in_headline:
                            line = next(file).strip()
                            if line.startswith("</HEADLINE>"):
                                in_headline = False
                            else:
                                headline_parts.append(re.sub(r'<.*?>', '', line))
                        #document['headline'] = ' '.join(headline_parts).strip()
                    elif line.startswith("<BYLINE>"):
                        in_byline = True
                        byline_parts = []
                        while in_byline:
                            line = next(file).strip()
                            if line.startswith("</BYLINE>"):
                                in_byline = False
                            else:
                                byline_parts.append(re.sub(r'<.*?>', '', line))
                        #document['byLine'] = ' '.join(byline_parts).strip()
                    elif line.startswith("<TEXT>"):
                        in_text = True
                        text_parts = []
                        while in_text:
                            line = next(file).strip()
                            if line.startswith("</TEXT>"):
                                in_text = False
                            else:
                                text_parts.append(re.sub(r'<.*?>', '', line))
                        document['text'] = ' '.join(text_parts).strip()

                    # End of a document
                    elif line.startswith("</DOC>"):
                        documents.append(document)
                        document = {}

        except Exception as ex:
            print(f"Failed to read file {file_path} with both UTF-8 and ISO-8859-1 encodings.")
            print(f"Error: {ex}")

    return documents


In [26]:
class DocumentBatchReader:
    def __init__(self, file_paths, batch_size=g_batch_size):
        self.file_paths = file_paths
        self.batch_size = batch_size
        self.current_file_index = 0

    def _read_all_from_file(self, file_path):
        # Read all documents from a single file
        return read_files_dataset(file_path)

    def get_next_batch(self):
        # Initialize an empty list to store batches
        batch = []
        # Get up to 4 files in a single batch
        for _ in range(4):
            if self.current_file_index < len(self.file_paths):
                file_path = self.file_paths[self.current_file_index]
                print(file_path)
                batch.extend(self._read_all_from_file(file_path))
                # Move to the next file for the next batch call
                self.current_file_index += 1
            else:
                break  # No more files to read
        print(len(batch))
        # Return batch if there are any files in it; otherwise, return None
        return batch if batch else None



In [31]:
import os
import json
import pandas as pd

# List all your files (assuming they are XML files and stored in a directory)
#file_paths = [f"./drive/MyDrive/Data/fbis/{file_name}" for file_name in os.listdir('./drive/MyDrive/Data/fbis/') if not file_name.endswith('.txt')]
file_paths = [f"./Data/latimes/{file_name}" for file_name in os.listdir('./Data/latimes/') if not file_name.endswith('.txt')]
print(file_paths)

# Create a batch reader instance
batch_reader = DocumentBatchReader(file_paths, batch_size=g_batch_size)
output_file_embeddings = './Embeddings/latimes/latimes-embeddings_'
output_file_documents = './Embeddings/latimes/latimes-documents_'
counter = 1

# Prepare lists to store documentIDs and texts separately
document_ids_data = []
texts_data = []

while True:
    batch = batch_reader.get_next_batch()
    if batch:
        # Step 1: Split the documents into two lists
        document_ids = []
        texts=[]
        for doc in batch:
            document_ids.append(doc['documentID'])
            if 'text' in doc:
                texts.append(doc['text'])
            else:
                texts.append("")
        print("ABC",len(texts))
        print("GETTING EMBEDDINGS")
        # Step 2: Get embeddings for the text batch
        embeddings = get_document_embedding(texts)
        print("GETTING EMBEDDINGS TAKEN")
        # Step 3: Store documentID and embedding separately
        document_ids_data.extend(document_ids)
        texts_data.extend(embeddings)
        print(len(document_ids_data),len(texts_data))
        # Save to files if the data exceeds 10,000 entries
        if len(document_ids_data) > 10000:
            # Save documentIDs
            output_file_ids = output_file_documents + str(counter) + '.txt'
            with open(output_file_ids, 'w') as f:
                for doc in document_ids_data:
                  f.write(doc+"\n")

            # Save embeddings
            output_file_texts = output_file_embeddings + str(counter) + '.txt'
            with open(output_file_texts, 'w') as f:
                for doc in texts_data:
                  f.write(" ".join(map(str, doc)) + "\n")

            print(f"Document IDs saved to {output_file_ids}.")
            print(f"Embeddings saved to {output_file_texts}.")

            # Increment counter and clear lists
            counter += 1
            document_ids_data = []
            texts_data = []
    else:
        # Save documentIDs
        output_file_ids = output_file_documents + str(counter) + '.txt'
        with open(output_file_ids, 'w') as f:
            for doc in document_ids_data:
                  f.write(doc+"\n")

        # Save embeddings
        output_file_texts = output_file_embeddings + str(counter) + '.txt'
        with open(output_file_texts, 'w') as f:
            for doc in texts_data:
                  f.write(" ".join(map(str, doc)) + "\n")

        print(f"Document IDs saved to {output_file_ids}.")
        print(f"Embeddings saved to {output_file_texts}.")
        print("No more documents available.")
        break


['./Data/latimes/la071790', './Data/latimes/la062989', './Data/latimes/la090890', './Data/latimes/la122989', './Data/latimes/la020790', './Data/latimes/la032890', './Data/latimes/la021689', './Data/latimes/la091989', './Data/latimes/la082790', './Data/latimes/la070689', './Data/latimes/la111390', './Data/latimes/la051390', './Data/latimes/la110289', './Data/latimes/la050289', './Data/latimes/la091389', './Data/latimes/la032290', './Data/latimes/la122389', './Data/latimes/la062389', './Data/latimes/la090290', './Data/latimes/la012690', './Data/latimes/la050889', './Data/latimes/la110889', './Data/latimes/la051990', './Data/latimes/la042789', './Data/latimes/la102789', './Data/latimes/la111990', './Data/latimes/la010990', './Data/latimes/la112789', './Data/latimes/la101990', './Data/latimes/la041990', './Data/latimes/la052789', './Data/latimes/la100889', './Data/latimes/la040889', './Data/latimes/la011889', './Data/latimes/la072389', './Data/latimes/la080290', './Data/latimes/la022290', 

In [1]:
for doc in batch:
    print(doc)


NameError: name 'batch' is not defined

In [ ]:
def cosine_similarity(vec1, vec2):
    print(np.shape(vec1),np.shape(vec2))
    # Calculate the dot product of the vectors
    dot_product = np.dot(vec1, vec2)
    # Calculate the L2 norms (magnitudes) of the vectors
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    # Compute cosine similarity
    return dot_product / (norm_vec1 * norm_vec2)

In [ ]:
cosine_similarity(embeddings[0].squeeze(),query_embeddings)

(768,) (768,)


np.float32(0.75074565)

In [3]:
import os

def count_doc_tags_in_file(file_path):
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        content = file.read()
    return content.count('<DOC>')


def count_doc_tags_in_folder(folder_path):
    total_count = 0
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            total_count += count_doc_tags_in_file(file_path)
    return total_count

folder_path = './Data/fbis/'  # Replace with your folder path
total_doc_tags = 0
total_doc_tags += count_doc_tags_in_folder(folder_path)
print(f"Total number of <DOC> tags: {total_doc_tags}")


Total number of <DOC> tags: 130474


In [ ]:
import json
import os
def count_document_ids(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    # Try to parse the content as a list of JSON objects
    try:
        # Load the content as a list (it should be a list of dictionaries or lists of dictionaries)
        data = json.loads(content)
        # Initialize a counter for documentID
        document_id_count = 0
        data = list(data)
        print(data[0])
        # Iterate over the structure and count occurrences of "documentID"
        if isinstance(data, list):  # Ensure the root structure is a list
            for item in data:
                if isinstance(item, list):  # Nested lists
                    for sub_item in item:
                        if isinstance(sub_item, dict) and 'documentID' in sub_item:
                            document_id_count += 1
                elif isinstance(item, dict) and 'documentID' in item:
                    document_id_count += 1

        return document_id_count

    except json.JSONDecodeError:
        print("Error parsing JSON.")
        return 0
count = 0
# Example usage for a single file
for i in os.listdir("Embeddings/fbis/"):
    file_path = 'Embeddings/fbis/'  + i
    count += count_document_ids(file_path)
    print(f"Number of documentID's in the file: {count}")



[


NameError: name 'content' is not defined

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.3 MB 6.3 MB/s eta 0:00:01
     |████████████████████████████████| 346 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 508 kB 5.6 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import json
count = 0
# Load the file content
for i in os.listdir('./Embeddings/fbis/'):
    with open('./Embeddings/fbis/' + i, 'r') as file:
        content = file.read()

    # Count occurrences of "documentID"
    count += content.count("documentID")
print(count)

133108
130474

133108
